In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("Resources/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


### Drop Additional Columns

In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=['EIN', 'NAME','STATUS'])
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,100000-499999,N,142590,1


In [3]:
# Look at APPLICATION_TYPE value counts for binning
value_counts = application_df['APPLICATION_TYPE'].value_counts()
value_counts

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

### Change bin ranges

In [4]:
# Determine which values to replace if counts are less than ...?
replace_application = value_counts[value_counts < 100].index.tolist()

# Replace in dataframe
for app in replace_application:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(app,"Other")
    
# Check to make sure binning was successful
application_df.APPLICATION_TYPE.value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
T9         156
Other      120
Name: APPLICATION_TYPE, dtype: int64

In [5]:
# Look at CLASSIFICATION value counts for binning
classifiaction_count = application_df['CLASSIFICATION'].value_counts()
classifiaction_count

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [6]:
# Determine which values to replace if counts are less than ..?
replace_class = classifiaction_count[classifiaction_count < 300].index.tolist()

# Replace in dataframe
for cls in replace_class:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(cls,"Other")
    
# Check to make sure binning was successful
application_df.CLASSIFICATION.value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1484
C7000      777
Name: CLASSIFICATION, dtype: int64

In [7]:
application_cat = ["APPLICATION_TYPE","AFFILIATION","CLASSIFICATION","USE_CASE","ORGANIZATION","INCOME_AMT","SPECIAL_CONSIDERATIONS"]

In [8]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df[application_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(application_cat)
encode_df.head()

,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,APPLICATION_TYPE_T9,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [9]:
# Merge one-hot encoded features and drop the originals
application_df = application_df.merge(encode_df,left_index=True, right_index=True)
application_df = application_df.drop(application_cat,1)
application_df.head()

C:\Users\aidan\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,5000,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,108590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,5000,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,6692,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,142590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [10]:
# Split our preprocessed data into our features and target arrays
y = application_df.IS_SUCCESSFUL
X = application_df.drop("IS_SUCCESSFUL",axis=1)

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [11]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

### Add additional hidden layers and activation functions

In [12]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_features = len(X_train_scaled[0])
hidden_layer1 = 100
hidden_layer2 = 50
hidden_layer3 = 30
hidden_layer4 = 20

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_layer1, input_dim=input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_layer2, activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_layer3, activation="linear"))

# Fourth hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_layer4, activation="tanh"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 100)               4500      
_________________________________________________________________
dense_1 (Dense)              (None, 50)                5050      
_________________________________________________________________
dense_2 (Dense)              (None, 30)                1530      
_________________________________________________________________
dense_3 (Dense)              (None, 20)                620       
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 21        
Total params: 11,721
Trainable params: 11,721
Non-trainable params: 0
_________________________________________________________________


In [13]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

### Add number of Epochs

In [14]:
# Train the model
fit_model = nn.fit(X_train,y_train,epochs=120)

Epoch 1/120
804/804 [==============================] - 1s 591us/step - loss: 0.7005 - accuracy: 0.5226
Epoch 2/120
804/804 [==============================] - 0s 585us/step - loss: 0.6915 - accuracy: 0.53210s - loss: 0.6915 - accuracy: 
Epoch 3/120
804/804 [==============================] - 0s 569us/step - loss: 0.6914 - accuracy: 0.5319
Epoch 4/120
804/804 [==============================] - 0s 570us/step - loss: 0.6917 - accuracy: 0.5309
Epoch 5/120
804/804 [==============================] - 0s 571us/step - loss: 0.6914 - accuracy: 0.5321
Epoch 6/120
804/804 [==============================] - 0s 573us/step - loss: 0.6914 - accuracy: 0.5311
Epoch 7/120
804/804 [==============================] - 0s 571us/step - loss: 0.6914 - accuracy: 0.5318
Epoch 8/120
804/804 [==============================] - 0s 575us/step - loss: 0.6915 - accuracy: 0.5311
Epoch 9/120
804/804 [==============================] - 0s 574us/step - loss: 0.6913 - accuracy: 0.5313
Epoch 10/120
804/804 [=====================

804/804 [==============================] - 0s 566us/step - loss: 0.6916 - accuracy: 0.5321
Epoch 80/120
804/804 [==============================] - 1s 656us/step - loss: 0.6915 - accuracy: 0.5321
Epoch 81/120
804/804 [==============================] - 1s 648us/step - loss: 0.6917 - accuracy: 0.5305
Epoch 82/120
804/804 [==============================] - 0s 596us/step - loss: 0.6915 - accuracy: 0.5306
Epoch 83/120
804/804 [==============================] - 1s 662us/step - loss: 0.6916 - accuracy: 0.5321
Epoch 84/120
804/804 [==============================] - 0s 602us/step - loss: 0.6915 - accuracy: 0.5316
Epoch 85/120
804/804 [==============================] - 1s 645us/step - loss: 0.6917 - accuracy: 0.5297
Epoch 86/120
804/804 [==============================] - 1s 693us/step - loss: 0.6915 - accuracy: 0.5299
Epoch 87/120
804/804 [==============================] - 1s 691us/step - loss: 0.6916 - accuracy: 0.5305
Epoch 88/120
804/804 [==============================] - 1s 693us/step - loss:

In [15]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.6909 - accuracy: 0.5332
Loss: 0.6909463405609131, Accuracy: 0.5331778526306152


In [16]:
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

In [17]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Create a callback that saves the model's weights every epoch
cp_callback = ModelCheckpoint(
    
    filepath=checkpoint_path, #the checkpoint directory and file structure we defined previously
    
    verbose=1, #we'll be notified when a checkpoint is being saved to the directory
    
    save_weights_only=True, #saving the full model each time can fill up a hard drive very quickly; 
                            #this ensures that the checkpoint files take up minimal space
    
    save_freq='epoch',

   period = 5
    
)  #checkpoints will be saved every 5 epoch

# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=200,callbacks=[cp_callback])

# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/200
804/804 [==============================] - 1s 596us/step - loss: 0.5631 - accuracy: 0.7249
Epoch 2/200
804/804 [==============================] - 0s 585us/step - loss: 0.5540 - accuracy: 0.7295
Epoch 3/200
804/804 [==============================] - 0s 585us/step - loss: 0.5516 - accuracy: 0.7299
Epoch 4/200
804/804 [==============================] - 0s 601us/step - loss: 0.5500 - accuracy: 0.7302
Epoch 5/200
804/804 [==============================] - 0s 600us/step - loss: 0.5492 - accuracy: 0.7323

Epoch 00005: saving model to checkpoints\weights.05.hdf5
Epoch 6/200
804/804 [==============================] - 0s 605us/step - loss: 0.5486 - accuracy: 0.7325
Epoch 7/200
804/804 [==============================] - 0s 574us/step - loss: 0.5478 - accuracy: 0.7329
Epoch 8/200
804/804 [==============================] - 0s 574us/step - loss: 0.5472 - accuracy: 0.7327
Epoch 9/200
804/804 [==============================] - 0s 574us/step - loss: 0.5462 - accuracy: 0.7346
Epoch 10/200
80

804/804 [==============================] - 0s 578us/step - loss: 0.5383 - accuracy: 0.7386
Epoch 72/200
804/804 [==============================] - 0s 568us/step - loss: 0.5376 - accuracy: 0.7397
Epoch 73/200
804/804 [==============================] - 0s 579us/step - loss: 0.5381 - accuracy: 0.7382
Epoch 74/200
804/804 [==============================] - 0s 574us/step - loss: 0.5383 - accuracy: 0.7360
Epoch 75/200
804/804 [==============================] - 0s 574us/step - loss: 0.5388 - accuracy: 0.7379

Epoch 00075: saving model to checkpoints\weights.75.hdf5
Epoch 76/200
804/804 [==============================] - 0s 578us/step - loss: 0.5396 - accuracy: 0.7386
Epoch 77/200
804/804 [==============================] - 0s 576us/step - loss: 0.5378 - accuracy: 0.7389
Epoch 78/200
804/804 [==============================] - 0s 568us/step - loss: 0.5374 - accuracy: 0.7400
Epoch 79/200
804/804 [==============================] - 0s 574us/step - loss: 0.5386 - accuracy: 0.7390
Epoch 80/200
804/80

804/804 [==============================] - 0s 573us/step - loss: 0.5375 - accuracy: 0.7393
Epoch 142/200
804/804 [==============================] - 0s 568us/step - loss: 0.5374 - accuracy: 0.7401
Epoch 143/200
804/804 [==============================] - 0s 573us/step - loss: 0.5384 - accuracy: 0.7381
Epoch 144/200
804/804 [==============================] - 0s 575us/step - loss: 0.5378 - accuracy: 0.7382
Epoch 145/200
804/804 [==============================] - 0s 576us/step - loss: 0.5377 - accuracy: 0.7386

Epoch 00145: saving model to checkpoints\weights.145.hdf5
Epoch 146/200
804/804 [==============================] - 0s 574us/step - loss: 0.5370 - accuracy: 0.7397
Epoch 147/200
804/804 [==============================] - 0s 576us/step - loss: 0.5359 - accuracy: 0.7401
Epoch 148/200
804/804 [==============================] - 0s 575us/step - loss: 0.5377 - accuracy: 0.7401
Epoch 149/200
804/804 [==============================] - 0s 573us/step - loss: 0.5386 - accuracy: 0.7387
Epoch 150/

In [18]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity_Optimization.h5")

In [19]:
# Import the model to a new object
nn_imported = tf.keras.models.load_model('AlphabetSoupCharity_Optimization.h5')

In [20]:
# Evaluate the completed model using the test data
model_loss, model_accuracy = nn_imported.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5616 - accuracy: 0.7228
Loss: 0.5616450309753418, Accuracy: 0.7227988243103027
